In [27]:
import cv2
import numpy as np

import os
import os.path

RESOLUTION = (32, 8)
TARGET_FPS = 12

INPUT_FILE = "video/processing/badapple.mp4"
FRAMES_DIR = "video/processing/frames"

In [28]:
video = cv2.VideoCapture(INPUT_FILE)

host_framerate = video.get(cv2.CAP_PROP_FPS)
frame_skip = host_framerate // TARGET_FPS

prev_frame = None

bytes_estimate = 0
byte_array = []
range_compressed = []

raw_framedata_array = []

cur_frame = frames_written = 0
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    if cur_frame % frame_skip == 0:
        frame = cv2.resize(frame, RESOLUTION, interpolation=cv2.INTER_NEAREST_EXACT)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        _, frame = cv2.threshold(frame, 127, 255, cv2.THRESH_BINARY)

        if cur_frame != 0:
            frame_xored = cv2.bitwise_xor(frame, prev_frame)
            bytes_estimate += np.count_nonzero(frame_xored)

            frame_changed = []
            tmp_range = []
            for y, row in enumerate(frame_xored):
                for x, val in enumerate(row):
                    if val > 127:
                        v_conv = x<<3 | y
                        if v_conv not in [0,1]:
                            frame_changed.append(v_conv)
            frame_changed = sorted(frame_changed)
            
            writing = []
            r = []
            for k, v in enumerate(frame_changed):
                if len(r) != 0 and r[-1]+1 != v:
                    if len(r) <= 3:
                        writing += r
                    else:
                        writing += [1, r[0], r[-1]]
                    r = [v]
                else:
                    r += [v]
            writing += r

            range_compressed += writing + [0]
            byte_array += frame_changed + [0]

            # for x in range(RESOLUTION[0]):
                # strip_one = 0
                # strip_two = 1
                # for y in range(8):
                    # first strip
                    # val_s_one = frame[y][x]
                    # val_s_two = frame[y+8][x]
                    
                    # if val_s_one > 127:
                    #     strip_one |= 1<<y
                    # if val_s_two > 127:
                    #     strip_two |= 1<<y
                # raw_framedata_array.append(strip_one)
                # raw_framedata_array.append(strip_two)

            cv2.imwrite(f"{FRAMES_DIR}/frame-{frames_written:04d}.jpg", frame_xored)
            frames_written += 1
        prev_frame = frame
    cur_frame += 1

video.release()

In [29]:
print(len(range_compressed), len(byte_array))

39683 49006


In [30]:
# convert bytes to c code
with open("dump.txt", 'w') as f:
    for v in range_compressed:
        if v == 0:
            f.write(f"{v},\n")
        else:
            f.write(f"{v},")